In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import pandas as pd
from collections import defaultdict

In [2]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
with open('combined_links_df.pickle', 'rb') as read_links:
    links_df = pickle.load(read_links)

In [4]:
with open('ppg.pickle', 'rb') as read_details:
    ppg = pickle.load(read_details)

In [5]:
links_df.set_index('Link',drop=False,inplace=True,verify_integrity=True)

In [6]:
links_df.head()

Name  \
Link                                                         
12th-man                             12th Man Freddy Funko   
1950-s-elvis                                  1950's Elvis   
1950-s-elvis-metallic-gold    1950's Elvis (Metallic Gold)   
1970-charger-w-dom-toretto  1970 Charger w/Dom Toretto #17   
1970-s-elvis                                  1970's Elvis   

                                                  Link  
Link                                                    
12th-man                                      12th-man  
1950-s-elvis                              1950-s-elvis  
1950-s-elvis-metallic-gold  1950-s-elvis-metallic-gold  
1970-charger-w-dom-toretto  1970-charger-w-dom-toretto  
1970-s-elvis                              1970-s-elvis

In [7]:
scrape = pd.DataFrame.from_dict(ppg, orient='index',columns=['Estimated_Value','Category_Names','Category_Values','Production_Status'])

In [33]:
for i in scrape['Category_Values'][:10]:
    print(i)

[]
['From: 2011', 'Plastic', 'Male', 'Black, Silver, Blue, Pink', '1', 'No', 'English', '']
['Plastic', 'Male', 'Black, Silver, Gold', '1', 'No', 'English', '']
['From: 2016', 'Plastic', 'Male', 'Black, White, Cadet Blue', '1', 'No', 'English', '']
['Plastic', 'Male', 'Black, White, Red, Gold', '1', 'No', 'English', '']
['Plastic', 'Male', 'Black, White, Red, Gold', '1', 'No', 'English', '']
['From: 2017', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Male', 'Black, White, Lime Green, Sky Blue', 'Bobblehead', 'Weapons', 'Box', 'No', 'English', '']
['From: 2017', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Male', 'Black, White, Lime Green, Sky Blue', 'Bobblehead', 'Weapons', 'Box', 'No', 'English', '']
['From: 2016', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Black, Gray, Dark Gray, Bronze, Dark Cyan', 'Bobblehead', 'Base', 'Box', 'No', 'English', '']
['From: 2016', 'Plastic, Plastic -> PVC/Vinyl', 'Black, Gray, Dark Gray, Bronze, Dark Cyan', 'Bobblehead', 'No', 'English', '']


In [34]:
for i in scrape['Category_Names'][:10]:
    print(i)

[]
['Produced:', 'Material:', 'Gender:', 'Color:', 'Number of Points of Articulation:', 'Blind Packaging?:', 'Package Language:']
['Material:', 'Production Status:', 'Gender:', 'Color:', 'Number of Points of Articulation:', 'Blind Packaging?:', 'Package Language:']
['Produced:', 'Material:', 'Production Status:', 'Gender:', 'Color:', 'Number of Points of Articulation:', 'Blind Packaging?:', 'Package Language:']
['Material:', 'Gender:', 'Color:', 'Number of Points of Articulation:', 'Blind Packaging?:', 'Package Language:']
['Material:', 'Production Status:', 'Gender:', 'Color:', 'Number of Points of Articulation:', 'Blind Packaging?:', 'Package Language:']
['Produced:', 'Material:', 'Production Status:', 'Scale:', 'Gender:', 'Color:', 'Features:', 'Accessories:', 'Packaging:', 'Blind Packaging?:', 'Package Language:']
['Produced:', 'Material:', 'Production Status:', 'Scale:', 'Gender:', 'Color:', 'Features:', 'Accessories:', 'Packaging:', 'Blind Packaging?:', 'Package Language:']
['Pro

In [8]:
for link in links_df['Link']:
    scrape.loc[link]['Category_Names'] = [x for x in scrape.loc[link]['Category_Names'] if (x[0]!='$')]

In [35]:
for link in links_df['Link']:
    try:
        scrape.loc[link]['Category_Values'].pop()
    except IndexError:
        continue

In [36]:
scrape['Category_Values']

12th-man                                                                                       []
1950-s-elvis                                    [From: 2011, Plastic, Male, Black, Silver, Blu...
1950-s-elvis-metallic-gold                      [Plastic, Male, Black, Silver, Gold, 1, No, En...
1970-charger-w-dom-toretto                      [From: 2016, Plastic, Male, Black, White, Cade...
1970-s-elvis                                    [Plastic, Male, Black, White, Red, Gold, 1, No...
                                                                      ...                        
zen-oh-galactic-toys-exclusive                  [From: May 2018, China, PVC/Vinyl, Male, Black...
zombie-venom-664-bobble-head-special-edition    [From: 2020, Viet Nam, PVC/Vinyl, Male, Black,...
venom-marvel-zombies                            [From: 2020, Viet Nam, PVC/Vinyl, Male, Black,...
zorak-sdcc                                      [From: 2016, China, PVC/Vinyl, Male, Black, Ye...
zorak               

In [ ]:
scrape[scrape['Category_Names']==[]]

In [ ]:
scrape[scrape['Category_Values']==[]]

In [9]:
missing_prices = scrape[scrape['Estimated_Value'].isnull()].index

In [ ]:
price_check = defaultdict(str)
db = 'https://www.hobbydb.com/marketplaces/poppriceguide/catalog_items/'
driver = webdriver.Chrome(chromedriver)
for i in missing_prices:
    url = db + i
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    price = soup.find('div', class_='value ng-binding')
    if price:
        price = price.text.strip()
    price_check[i] = price
    time.sleep(4)

In [ ]:
print(price_check)

In [ ]:
#with open('missing_est_values.pickle', 'wb') as missing_estimated_values_pickle:
    #pickle.dump(price_check, missing_estimated_values_pickle)

In [10]:
with open('missing_est_values.pickle','rb') as read_missing_values:
    price_check = pickle.load(read_missing_values)

In [11]:
scrape[scrape['Estimated_Value'].isnull()]

Estimated_Value  \
12th-man                                                      None   
70s-wanda                                                     None   
abraham-prototype                                             None   
ace-levy                                                      None   
adam-banks                                                    None   
...                                                            ...   
vision-71-metallic-bobble-head-underground-toys               None   
vision-71-phasing-bobble-head-underground-toys                None   
white-ranger-jason-david-frank-autograph                      None   
white-ranger-action-pose-jason-david-frank-auto...            None   
zen-oh-7a549f22-88de-4486-b887-6a3a8e538b19                   None   

                                                                                       Category_Names  \
12th-man                                                                                           []   
70s-wanda                                           [Produced:, Material:, Production Status:, Sca...   
abraham-prototype                                   [Produced:, Material:, Production Status:, Gen...   
ace-levy                                            [Produced:, Material:, Production Status:, Sca...   
adam-banks                                          [Produced:, Material:, Production Status:, Sca...   
...                                                                                               ...   
vision-71-metallic-bobble-head-underground-toys     [Produced:, Material:, Production Status:, Gen...   
vision-71-phasing-bobble-head-underground-toys      [Produced:, Material:, Production Status:, Gen...   
white-ranger-jason-david-frank-autograph            [Produced:, # Produced:, Material:, Production...   
white-ranger-action-pose-jason-david-frank-auto...  [Produced:, # Produced:, Material:, Production...   
zen-oh-7a549f22-88de-4486-b887-6a3a8e538b19         [Produced:, Made In:, Material:, Production St...   

                                                                                      Category_Values  \
12th-man                                                                                           []   
70s-wanda                                           [From: December 2020, Plastic, 3.75", Female, ...   
abraham-prototype                                   [From: 2015, Plastic, Male, Gray, Army Brown, ...   
ace-levy                                            [From: December 2020, Plastic, 3.75", Male, Bl...   
adam-banks                                          [From: 2020, Plastic, 3.75", Male, Black, Whit...   
...                                                                                               ...   
vision-71-metallic-bobble-head-underground-toys     [From: 2015, Plastic, Male, Metallic, Silver, ...   
vision-71-phasing-bobble-head-underground-toys      [From: 2015, Plastic, Male, Transparent, Gold,...   
white-ranger-jason-david-frank-autograph            [From: August 2020, 1, Plastic, 3.75", Male, B...   
white-ranger-action-pose-jason-david-frank-auto...  [From: August 2020, 1, Plastic, 3.75", Male, B...   
zen-oh-7a549f22-88de-4486-b887-6a3a8e538b19         [From: May 2018, China, PVC/Vinyl, 3.75", Male...   

                                                                                   Production_Status  
12th-man                                            [Production Status:, Exclusive, Limited Edition]  
70s-wanda                                                             [Production Status:, Standard]  
abraham-prototype                                                    [Production Status:, Prototype]  
ace-levy                                                              [Production Status:, Standard]  
adam-banks                                                            [Production Status:, Standard]  
...                                                                   

In [12]:
for i in missing_prices:
    scrape.loc[i]['Estimated_Value'] = price_check[i]

In [ ]:
scrape[scrape['Estimated_Value'].isnull()]

In [18]:
for pop in links_df['Link']:
    for column in scrape.loc[i]['Category_Names']):
        scrape.loc[i][column] = 

SyntaxError: invalid syntax (<ipython-input-18-5e883fe53a81>, line 2)

In [ ]:
names_values_df = pd.DataFrame(scrape.loc['1950-s-elvis']['Category_Values'],columns=scrape.loc['1950-s-elvis']['Category_Names'])

In [ ]:
names_values_df

In [46]:
len(scrape.loc['1950-s-elvis']['Category_Values'])

7

In [15]:
len(scrape.loc['1950-s-elvis']['Category_Names'])

7

In [16]:
for i in links_df['Link']:
    if len(scrape.loc[i]['Category_Names'])!=len(scrape.loc[i]['Category_Values']):
        print(i)

1950-s-elvis
1970-s-elvis
aargh
alan-carlos
alan-dutch-schaefer-vinyl-art-toys
alexander-nitrokoff
alexander-nitrokoff-dynamite
alexis-rose
alice-angel
alien-metallic
all-for-one
all-for-one-battle-hand
alternate-universe-zoidberg
amare-stoudemire
amy-farrah-fowler-5788fc10-bc48-4c5d-a93f-96b7cb81c8a5
amy-farrah-fowler-tiara
vaporean
andy-dwyer-johnny-karate
andy-dwyer-mouse-rat
angel-first-appearance
angel-82af1c34-a001-4e81-b689-e9e1675fd8a8
angela-with-sprinkles
anger-flames-sdcc
animal-746cc12a-1c19-4e5b-9770-6877708a8d40
annie-leonhart-38399b18-3665-4ce8-949e-a258e1b7197a
ant-man-classic-sdcc
ant-man-gold-chrome
ape-soldier
aquarius-eca4836e-6f69-41c7-806c-bae94c30b9c0
arach-knight
archangel
archie-andrews-vinyl-art-toys
aries-aa5e1030-851a-470a-a997-930ec2f44e9c
astrid-6ced62b6-16aa-4f9d-85bd-9fe2301df593
aughra-age-of-resistance
aurora-maleficent-movie
aveline-de-grandpre
avengers-assemble-hawkeye
azog
baby-sinclair-vinyl-art-toys
bad-ape
badtz-maru-aa9e080d-4971-4eaa-8fbc-dfa48

golden-frieza-black-eyes-sdcc
gollum
gonzo
good-luck-bear-vinyl-art-toys
goofy-0ea46796-b7e2-4869-b239-0bd47ea61879
gorilla-grodd
gorilla-grodd-summer-convention
gossamer-flocked
grace-adler
great-ape-goku
greef-karga
green-lantern-john-stewart
greg-jenko
reaper-homer
groot-flocked-18-inch
groot-gamer
groot-gamer-standing
groot-holiday-standing
groot-toddler-with-lights
groot-infinity-war-with-stormbreaker
groot-vol-2-10-inch
groot-vol-2-candy-bowl
groot-vol-2-jumpsuit
groot-vol-2-jumpsuit-patch
groot-with-lights
groot-wood-deco
gru
grumpy-08435c42-79a4-4059-b756-5a2996ac87a1
grumpy-bear-vinyl-art-toys
grunt
hades-on-throne
hadji
hammerhead-eae3dccf-778d-4dcc-bef9-bc1339d8e632
han-solo-the-force-awakens
han-solo-the-force-awakens-w-bowcaster-sdcc
hanuman-polystone
happy-gilmore-hockey-stick
harley-quinn-bombshells-vintage
harry-hermione-ron-riding-gringotts-dragon
hazel-w-mask
heartless-2d87d4e0-e8b8-430f-8174-a0c2d7adbcc1
heat-miser
hector-bellerin
heimdall-dark-world
hello-kitty-9ed4

r2-d2-dagobah
rafiki
rafiki-with-simba
raj-koothrappali-4baee660-2a0c-47f2-9899-359ad82092ae
raj-koothrappali-with-martini
randall-boggs
ranger-smith
rapunzel-dancing-glitter
rapunzel-w-frying-pan-pascal-red-summer-convention
rat-fink-gray
red-hulk-f7810583-e22d-41e5-b5b5-5ed035bf0edd
red-ranger-419bdbc2-81ab-42f1-aa28-5c6e21700f80
regina-w-fireball
renly-baratheon
reverse-flash
rian
ricky-rubio
rigby-d0e454a3-65e7-4a10-bbf7-d0f0e200be9f
ringo-starr-yellow-submarine
robbie-sinclair-vinyl-art-toys
robb-stark
robby-the-robot
dc-super-heroes-robin-chase-02
robocop-53153fee-2173-4e8a-a2f9-4c14037a3658
rocketeer
rocket-raccoon-nova-potted-groot-sdcc
rocket-vol-2
rocky-6fddf9a6-1701-414c-8724-fd79da06b2c2
rocky-balboa-390a6df4-4cf0-467f-868e-6f5241cebd74
roger-rabbit
rogue-x-men-20th
ronda-rousey
ron-puking-slugs-with-bucket
rorschach
rosa
roz-remix
rudolph
rumplestiltskin
rumplestiltskin-gold
russell-vinyl-art-toys
sadness-glitter-hair-sdcc
sagittarius
sailor-chibi-moon-57792d3f-a3df-46bf-a

captain-america-bucky-cap-sdcc
classic-sonic
boba-fett-concept-series-galactic-convention
boba-fett-concept-series
deadpool-venom-vinyl-art-toys
dug-cone-of-shame-sdcc
hardy-har-har-fall-convention
hardy-har-har
harley-quinn-with-mallet-suicide-squad-rebirth
izuku-midoriya-metallic-vinyl-art-toys
izuku-midoriya-vinyl-art-toys
jim-with-armor-red-nycc
jim-red-armor
lippy-the-lion
miles-morales-street-art-fall-convention
miles-morales-street-art
naruto-six-path
ned-vinyl-art-toys
scooby-doo-910-young-special-edition
young-scooby-doo
shaggy-911-young-special-edition
young-shaggy
silver-vinyl-art-toys
storm-59-glow-bobble-head-2014-supanova
storm-glow-in-the-dark
the-creeper-eccc
the-flash-unmasked-sdcc
the-flash-unmasked
toucan-cape
velociraptor-vinyl-art-toys
venomized-doctor-strange-glow-in-the-dark-vinyl-art-toys
venomized-doctor-strange-glow-in-the-dark
werewolf-631-popcultcha
werewolf-vinyl-art-toys
werewolf-bart-fall-convention
werewolf-bart-nycc
white-ranger-bb21eee9-a120-4c94-87ac-

In [40]:
sublinklist = links_df['Link'][:10]

In [41]:
print(sublinklist)

Link
12th-man                                                                                                       12th-man
1950-s-elvis                                                                                               1950-s-elvis
1950-s-elvis-metallic-gold                                                                   1950-s-elvis-metallic-gold
1970-charger-w-dom-toretto                                                                   1970-charger-w-dom-toretto
1970-s-elvis                                                                                               1970-s-elvis
1970-s-elvis-glow-in-the-dark-479dfff4-e35a-41ac-a66c-20b80117df2c    1970-s-elvis-glow-in-the-dark-479dfff4-e35a-41...
442nd-clone-trooper-celebration                                                         442nd-clone-trooper-celebration
442nd-clone-trooper                                                                                 442nd-clone-trooper
4-lom-58f0e626-d30e-4e19-aac8-e75f0

In [45]:
for i in sublinklist:
    print(scrape.loc[i]['Category_Values'])

[]
['From: 2011', 'Plastic', 'Male', 'Black, Silver, Blue, Pink', '1', 'No', 'English']
['Plastic', 'Male', 'Black, Silver, Gold', '1', 'No', 'English']
['From: 2016', 'Plastic', 'Male', 'Black, White, Cadet Blue', '1', 'No', 'English']
['Plastic', 'Male', 'Black, White, Red, Gold', '1', 'No', 'English']
['Plastic', 'Male', 'Black, White, Red, Gold', '1', 'No', 'English']
['From: 2017', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Male', 'Black, White, Lime Green, Sky Blue', 'Bobblehead', 'Weapons', 'Box', 'No', 'English']
['From: 2017', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Male', 'Black, White, Lime Green, Sky Blue', 'Bobblehead', 'Weapons', 'Box', 'No', 'English']
['From: 2016', 'Plastic, Plastic -> PVC/Vinyl', '3.75"', 'Black, Gray, Dark Gray, Bronze, Dark Cyan', 'Bobblehead', 'Base', 'Box', 'No', 'English']
['From: 2016', 'Plastic, Plastic -> PVC/Vinyl', 'Black, Gray, Dark Gray, Bronze, Dark Cyan', 'Bobblehead', 'No', 'English']


In [48]:
for i in links_df['Link']:
    scrape.loc[i]['Category_Values'] = [value for value in scrape.loc[i]['Category_Values'] if value]